# Malta Covid-19 Data Analysys

## Import Libraries

In [151]:
import pandas as pd
import numpy as np
import calendar
from functools import reduce
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

## Load Datasets

In [199]:
cases = pd.read_csv('../data/raw/COVID-19 Malta - Aggregate Data Set.csv')
tests = pd.read_csv('../data/raw/COVID-19 Malta - COVID Tests.csv')
vaccines = pd.read_csv('../data/raw/COVID-19 Malta - Vaccination Data.csv')
hospitalized = pd.read_csv('../data/raw/COVID-19 EU - Daily Hospitalized.csv')

## Group Covid Cases Dataset

In [172]:
cases_df = pd.DataFrame()
cases_df['Date'] = pd.to_datetime(cases['Date'], format='%d/%m/%Y')
cases_df = cases.groupby(cases_df.Date.dt.to_period('M')).agg({
    'New Cases': 'sum',
    'Total Cases': 'last',
    'Recovered': 'last',
    'Deaths': 'last',
    'Active Cases': 'last'
})

cases_df.insert(0, 'Date', cases_df.index.strftime('%m/%Y'))
cases_df.insert(1, 'Year', cases_df.index.year)
cases_df.insert(2, 'Month', cases_df.index.month)

cases_df = cases_df.reset_index(drop=True)

cases_df['Month'] = cases_df['Month'].apply(lambda x: calendar.month_abbr[x])

def get_diff(row, column):
    if row.name == 0:
        return row[column]
    return row[column] - cases_df.iloc[row.name - 1][column]

cases_df.insert(6, 'New Recovered', cases_df.apply(get_diff, axis=1, column='Recovered'))
cases_df.insert(8, 'New Deaths', cases_df.apply(get_diff, axis=1, column='Deaths'))

cases_df.rename(columns={
    'Recovered': 'Total Recovered',
    'Deaths': 'Total Deaths'
}, inplace=True)

cases_df

,Date,Year,Month,New Cases,Total Cases,Total Recovered,New Recovered,Total Deaths,New Deaths,Active Cases
0,03/2020,2020,Mar,169,169,2,2,0,0,167
1,04/2020,2020,Apr,296,465,351,349,4,4,110
2,05/2020,2020,May,153,618,534,183,9,5,75
3,06/2020,2020,Jun,52,670,640,106,9,0,21
4,07/2020,2020,Jul,154,824,665,25,9,0,150
5,08/2020,2020,Aug,1164,1883,1400,735,12,3,471
6,09/2020,2020,Sep,1175,3058,2562,1162,34,22,462
7,10/2020,2020,Oct,2984,6042,4155,1593,62,28,1825
8,11/2020,2020,Nov,3831,9873,7665,3510,137,75,2071
9,12/2020,2020,Dec,2901,12774,11121,3456,219,82,1434


## Group Tests Dataset

In [246]:
tests_df = pd.DataFrame({
    'Date': tests['Publication date'],
    'Tests in prev day': tests['NAA and rapid antigen tests in previous day'],
    'Total Tests': tests['Total NAA and rapid antigen tests']
})
tests_df = tests_df.apply(
    lambda row: 
        row if row.name == len(tests_df.index) - 1 
            else [
                row['Date'], 
                tests_df.iloc[row.name + 1]['Tests in prev day'], 
                tests_df.iloc[row.name + 1]['Total Tests']], 
    axis=1, 
    result_type='broadcast'
)
tests_df = tests_df.rename(columns={
    'Tests in prev day': 'New Tests'
})

positivity_rate = (cases['New Cases'] / tests_df['New Tests']) * 100
positivity_rate = positivity_rate.astype(np.float64)

tests_df['Date'] = pd.to_datetime(tests_df['Date'], format='%d/%m/%Y')
tests_df = tests_df.groupby(tests_df.Date.dt.to_period('M')).agg({
    'New Tests': 'sum',
    'Total Tests': 'last'
})
tests_df.insert(0, 'Date', tests_df.index.strftime('%m/%Y'))
tests_df = tests_df.reset_index(drop=True)
tests_df

,Date,New Tests,Total Tests
0,07/2020,14927,128747
1,08/2020,63762,192509
2,09/2020,64048,256557
3,10/2020,83176,339733
4,11/2020,91049,430782
5,12/2020,86754,517536
6,01/2021,98081,615617
7,02/2021,86493,702110
8,03/2021,108992,811102
9,04/2021,57961,869063


## Group Vaccines Dataset

In [174]:
vaccines_df = vaccines.rename(columns={
    'Date of Vaccination': 'Date',
    'Primary Vaccination': 'Total Primary Vaccination',
    'Received one dose': 'Total Received one dose'
})
vaccines_df['Date'] = pd.to_datetime(vaccines_df['Date'], format='%d/%m/%Y')
vaccines_df = vaccines_df.groupby(vaccines_df.Date.dt.to_period('M')).agg({
    'Total Vaccination Doses': 'last',
    'Total Primary Vaccination': 'last',
    'Total Received one dose': 'last',
    'Total Booster doses': 'last',
    'Total 2nd Booster doses': 'last',
    'Omicron booster doses': 'sum',
    'Total Omicron booster doses': 'last'
})
vaccines_df = vaccines_df.replace({np.nan: 0})
vaccines_df = vaccines_df.astype(np.int64)

vaccines_df.insert(0, 'Date', vaccines_df.index.strftime('%m/%Y'))
vaccines_df = vaccines_df.reset_index(drop=True)

def get_diff(row, column):
    if row.name == 0:
        return row[column]
    return row[column] - vaccines_df.iloc[row.name - 1][column]

vaccines_df.insert(2, 'New Vaccination Doses', vaccines_df.apply(get_diff, axis=1, column='Total Vaccination Doses'))
vaccines_df.insert(4, 'New Primary Vaccination', vaccines_df.apply(get_diff, axis=1, column='Total Primary Vaccination'))
vaccines_df.insert(6, 'New Received one dose', vaccines_df.apply(get_diff, axis=1, column='Total Received one dose'))
vaccines_df.insert(8, 'New Booster doses', vaccines_df.apply(get_diff, axis=1, column='Total Booster doses'))
vaccines_df.insert(10, 'New 2nd Booster doses', vaccines_df.apply(get_diff, axis=1, column='Total 2nd Booster doses'))

vaccines_df.insert(13, 'New Omicron booster doses', vaccines_df['Omicron booster doses'])
vaccines_df = vaccines_df.drop(columns=['Omicron booster doses'])

vaccines_df

,Date,Total Vaccination Doses,New Vaccination Doses,Total Primary Vaccination,New Primary Vaccination,Total Received one dose,New Received one dose,Total Booster doses,New Booster doses,Total 2nd Booster doses,New 2nd Booster doses,Total Omicron booster doses,New Omicron booster doses
0,01/2021,27759,27759,3948,3948,23811,23811,0,0,0,0,0,0
1,02/2021,79624,51865,28048,24100,51576,27765,0,0,0,0,0,0
2,03/2021,197383,117759,54578,26530,142805,91229,0,0,0,0,0,0
3,04/2021,335848,138465,107038,52460,228810,86005,0,0,0,0,0,0
4,05/2021,519994,184146,210486,103448,318743,89933,0,0,0,0,0,0
5,06/2021,670759,150765,325891,115405,359169,40426,0,0,0,0,0,0
6,07/2021,759818,89059,382255,56364,399690,40521,0,0,0,0,0,0
7,08/2021,797017,37199,412767,30512,413985,14295,0,0,0,0,0,0
8,09/2021,824327,27310,421713,8946,421602,7617,12071,12071,0,0,0,0
9,10/2021,879909,55582,428333,6620,430283,8681,52915,40844,0,0,0,0


## Hospital Dataset

In [175]:
malta_hospitalized = hospitalized.loc[hospitalized['country'] == 'Malta'].drop(columns=['year_week', 'source', 'url', 'country'])
malta_hospitalized = malta_hospitalized.rename(columns={'date': 'Date'})
malta_hospitalized['value'] = malta_hospitalized['value'].astype(np.int64)
malta_hospitalized['Date'] = pd.to_datetime(malta_hospitalized['Date'], format='%Y-%m-%d')

### Group Hosptialized Data

In [176]:
hospitalized_df = malta_hospitalized.loc[malta_hospitalized['indicator'] == 'Daily hospital occupancy'].drop(columns=['indicator'])
hospitalized_df = hospitalized_df.rename(columns={'value': 'Hospitalized'})
hospitalized_df = hospitalized_df.reset_index(drop=True)

hospitalized_df = hospitalized_df.groupby(hospitalized_df.Date.dt.to_period('M')).agg({
    'Hospitalized': 'last'
})
hospitalized_df.insert(0, 'Date', hospitalized_df.index.strftime('%m/%Y'))
hospitalized_df = hospitalized_df.reset_index(drop=True)
hospitalized_df

,Date,Hospitalized
0,03/2020,9
1,04/2020,4
2,05/2020,3
3,06/2020,0
4,07/2020,1
5,08/2020,7
6,09/2020,22
7,10/2020,53
8,11/2020,80
9,12/2020,66


### Group ICU Data

In [177]:
icu_df = malta_hospitalized.loc[malta_hospitalized['indicator'] == 'Daily ICU occupancy'].drop(columns=['indicator'])
icu_df = icu_df.rename(columns={'value': 'ICU'})
icu_df = icu_df.reset_index(drop=True)

icu_df = icu_df.groupby(icu_df.Date.dt.to_period('M')).agg({
    'ICU': 'last'
})
icu_df.insert(0, 'Date', icu_df.index.strftime('%m/%Y'))
icu_df = icu_df.reset_index(drop=True)
icu_df

,Date,ICU
0,03/2020,4
1,04/2020,1
2,05/2020,2
3,06/2020,0
4,07/2020,0
5,08/2020,0
6,09/2020,5
7,10/2020,7
8,11/2020,17
9,12/2020,10


## Combine Data

In [178]:
dataframes = [cases_df, tests_df, vaccines_df, hospitalized_df, icu_df]
data_df = reduce(
    lambda combined_data, df:
        pd.merge(
            left=combined_data,
            right=df,
            how='outer',
            on='Date'
        ),
    dataframes
)

# Add Positivity Rate
data_df.insert(3, 'Positivity Rate', (data_df['New Cases'] / data_df['New Tests']) * 100)

data_df

,Date,Year,Month,Positivity Rate,New Cases,Total Cases,Total Recovered,New Recovered,Total Deaths,New Deaths,...,Total Received one dose,New Received one dose,Total Booster doses,New Booster doses,Total 2nd Booster doses,New 2nd Booster doses,Total Omicron booster doses,New Omicron booster doses,Hospitalized,ICU
0,03/2020,2020,Mar,NaN,169,169,2,2,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,4
1,04/2020,2020,Apr,NaN,296,465,351,349,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1
2,05/2020,2020,May,NaN,153,618,534,183,9,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2
3,06/2020,2020,Jun,NaN,52,670,640,106,9,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
4,07/2020,2020,Jul,1.031688,154,824,665,25,9,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
5,08/2020,2020,Aug,1.825539,1164,1883,1400,735,12,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,0
6,09/2020,2020,Sep,1.834562,1175,3058,2562,1162,34,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,5
7,10/2020,2020,Oct,3.587573,2984,6042,4155,1593,62,28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0,7
8,11/2020,2020,Nov,4.207624,3831,9873,7665,3510,137,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,17
9,12/2020,2020,Dec,3.343938,2901,12774,11121,3456,219,82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.0,10


## Calculate Correlations

In [179]:
def calc_correlations(left: pd.Series, right: pd.Series):
    print("Pearson's correlation coefficient: ", left.corr(right, method='pearson'))
    print("Kendall's tau: ", left.corr(right, method='kendall'))
    print("Spearman's rank correlation coefficient: ", left.corr(right, method='spearman'))

## Total Cases, Recovered & Deaths

In [180]:
fig = px.line(
    data_df, 
    x='Date', 
    y=['Total Cases', 'Total Recovered', 'Total Deaths'], 
    title='Total Cases, Recovered & Deaths'
)
fig.update_layout(
    legend_title='Legend',
    yaxis_title='Cases'
)
fig.show()

### New Cases & Active Cases

In [181]:
fig = go.Figure(data=[
    go.Bar(name='Active Cases', x=data_df['Date'], y=data_df['Active Cases']),
    go.Bar(name='New Cases', x=data_df['Date'], y=data_df['New Cases'])
])
fig.update_layout(barmode='group')

fig.update_layout(
    title_text="New Cases & Active Cases"
)

fig.update_yaxes(title_text="Cases")

fig.show()

#### Probability Distribution of New Cases & Active Cases per Month

In [185]:
fig = go.Figure(data=[
    go.Histogram(x=data_df['Date'], y=data_df['New Cases'], histfunc='sum', histnorm='probability', name='New Cases'),
    go.Histogram(x=data_df['Date'], y=data_df['Active Cases'], histfunc='sum', histnorm='probability', name='Active Cases')
])

fig.update_layout(barmode='group')

fig.update_layout(
    title_text="Probability Distribution of New Cases & Active Cases per Month"
)

fig.update_yaxes(title_text="Probability")

fig.show()

#### Distribution of Daily New Cases

In [203]:
fig = ff.create_distplot(
    [cases['New Cases']], 
    ['New Cases'], 
    bin_size=[100],
    show_rug=False
)

fig.update_xaxes(title_text="Cases")
fig.update_layout(title_text='Distribution of Daily New Cases')

fig.show()

### Active Cases & Positivity Rate

In [187]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['Active Cases'],
        mode='lines+markers',
        name='Active Cases'
    )
)
fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['Positivity Rate'],
        mode='lines+markers',
        name='Positivity Rate'
    ),
    secondary_y=True
)

fig.update_layout(
    title_text="Active Cases & Positivity Rate %"
)

fig.update_yaxes(title_text="Cases", secondary_y=False)
fig.update_yaxes(title_text="Rate %", secondary_y=True)

fig.show()

#### Probability of Active Cases & Positivity Rate %

In [189]:
fig = go.Figure(data=[
    go.Histogram(x=data_df['Date'], y=data_df['Active Cases'], histfunc='sum', histnorm='probability', name='Active Cases'),
    go.Histogram(x=data_df['Date'], y=data_df['Positivity Rate'], histfunc='sum', histnorm='probability', name='Positivity Rate %')
])

fig.update_layout(barmode='group')

fig.update_layout(
    title_text="Probability of Active Cases & Positivity Rate %"
)

fig.update_yaxes(title_text="Probability")

fig.show()

### Tests, New Cases & Positivity Rate

In [60]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_traces([
    go.Bar(name='Tests', x=data_df['Date'], y=data_df['New Tests']),
    go.Bar(name='Cases', x=data_df['Date'], y=data_df['New Cases'])
])
fig.update_layout(barmode='group')

fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['Positivity Rate'],
        mode='lines+markers',
        name='Positivity Rate'
    ),
    secondary_y=True
)

fig.update_layout(
    title_text="Tests, New Cases & Positivity Rate %"
)

fig.update_yaxes(title_text="Cases / Tests", secondary_y=False)
fig.update_yaxes(title_text="Rate %", secondary_y=True)

fig.show()

#### Distribution of Daily Tests & New Cases

In [234]:
fig = ff.create_distplot(
    [tests['Total NAA and rapid antigen tests'].diff().dropna()], 
    ['Daily Tests'],
    bin_size=[1000],
    show_rug=False,
    show_hist=False
)

fig.update_xaxes(title_text="Tests")
fig.update_layout(title_text='Distribution of Daily Tests')

fig.show()


In [235]:
fig = ff.create_distplot(
    [cases['New Cases']], 
    ['Daily Cases'],
    bin_size=[100],
    show_rug=False,
    show_hist=False
)

fig.update_xaxes(title_text="Cases")
fig.update_layout(title_text='Distribution of Daily Cases')

fig.show()

#### Correlation of Daily Tests and Daily Cases

In [233]:
calc_correlations(tests['Total NAA and rapid antigen tests'], cases['New Cases'])

Pearson's correlation coefficient:  0.41628991931343157
Kendall's tau:  0.3645016862819882
Spearman's rank correlation coefficient:  0.5286085619911163


### Active Cases & Recovered

In [61]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=data_df['Date'],
        y=data_df['Active Cases'],
        name='Active Cases'
    )
)
fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['New Recovered'],
        mode='lines+markers',
        name='Recovered'
    )
)

fig.update_layout(
    title_text="Active Cases & Recovered"
)

fig.update_yaxes(title_text="Cases")

fig.show()

### Deaths, Positivity Rate

In [62]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['New Deaths'],
        mode='lines+markers',
        name='Deaths'
    )
)
fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['Positivity Rate'],
        mode='lines+markers',
        name='Positivity Rate'
    ),
    secondary_y=True
)

fig.update_layout(
    title_text="Deaths & Positivity Rate %"
)

fig.update_yaxes(title_text="Deaths", secondary_y=False)
fig.update_yaxes(title_text="Rate %", secondary_y=True)

fig.show()

#### Distribution of Deaths

In [255]:
fig = ff.create_distplot(
    [cases['Deaths'].diff().dropna()], 
    ['Daily Deaths'],
    show_rug=False,
    show_hist=False
)

fig.update_xaxes(title_text="Deaths")
fig.update_layout(title_text='Distribution of Daily Deaths')

fig.show()

#### Correlation between Positivity Rate % and Daily Deaths

In [257]:
calc_correlations(cases['Deaths'].diff(), positivity_rate)

Pearson's correlation coefficient:  0.23655013586896254
Kendall's tau:  0.36626558686861577
Spearman's rank correlation coefficient:  0.47716583701240944


### Hospitalizations, ICU & Positivity Rate

In [63]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_traces([
    go.Bar(name='Hospitalizations', x=data_df['Date'], y=data_df['New Hospitalized']),
    go.Bar(name='ICU', x=data_df['Date'], y=data_df['New ICU'])
])
fig.update_layout(barmode='group')

fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['Positivity Rate'],
        mode='lines+markers',
        name='Positivity Rate'
    ),
    secondary_y=True
)

fig.update_layout(
    title_text="Hospitalizations, ICU & Positivity Rate %"
)

fig.update_yaxes(title_text="Cases", secondary_y=False)
fig.update_yaxes(title_text="Rate %", secondary_y=True)

fig.show()

## Vaccinations

In [64]:
fig = go.Figure(data=[
    go.Bar(name='Received One Dose', x=data_df['Date'], y=data_df['New Received one dose']),
    go.Bar(name='Primary Vaccination', x=data_df['Date'], y=data_df['New Primary Vaccination']),
    go.Bar(name='Booster Doses', x=data_df['Date'], y=data_df['New Booster doses']),
    go.Bar(name='2nd Booster Doses', x=data_df['Date'], y=data_df['New 2nd Booster doses']),
    go.Bar(name='Omicron Booster Doses', x=data_df['Date'], y=data_df['New Omicron booster doses'])
])
fig.update_layout(barmode='stack')

fig.update_layout(
    title_text="Vaccinations by Type"
)

fig.update_yaxes(title_text="Doses")

fig.show()

### Vaccinations & Positivity Rate

In [65]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_traces([
    go.Bar(name='Received One Dose', x=data_df['Date'], y=data_df['New Received one dose']),
    go.Bar(name='Primary Vaccination', x=data_df['Date'], y=data_df['New Primary Vaccination']),
    go.Bar(name='Booster Doses', x=data_df['Date'], y=data_df['New Booster doses']),
    go.Bar(name='2nd Booster Doses', x=data_df['Date'], y=data_df['New 2nd Booster doses']),
    go.Bar(name='Omicron Booster Doses', x=data_df['Date'], y=data_df['New Omicron booster doses'])
])
fig.update_layout(barmode='stack')

fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['Positivity Rate'],
        mode='lines+markers',
        name='Positivity Rate',
        marker_color='black'
    ),
    secondary_y=True
)


fig.update_layout(
    title_text="Vaccinations & Positivity Rate %"
)

fig.update_yaxes(title_text="Doses", secondary_y=False)
fig.update_yaxes(title_text="Rate %", secondary_y=True)

fig.show()


### Vaccinations, Positivity Rate & Deaths

In [66]:
fig = go.Figure()

fig.add_traces([
    go.Bar(name='Received One Dose', x=data_df['Date'], y=data_df['New Received one dose']),
    go.Bar(name='Primary Vaccination', x=data_df['Date'], y=data_df['New Primary Vaccination']),
    go.Bar(name='Booster Doses', x=data_df['Date'], y=data_df['New Booster doses']),
    go.Bar(name='2nd Booster Doses', x=data_df['Date'], y=data_df['New 2nd Booster doses']),
    go.Bar(name='Omicron Booster Doses', x=data_df['Date'], y=data_df['New Omicron booster doses'])
])
fig.update_layout(barmode='stack')

fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['Positivity Rate'],
        mode='lines+markers',
        name='Positivity Rate',
        marker_color='black',
        yaxis='y2'
    )
)
fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['New Deaths'],
        mode='lines+markers',
        name='Deaths',
        marker_color='red',
        yaxis='y3'
    )
)

fig.update_layout(
    xaxis=dict(
        domain=[0, 0.95]
    ),
    yaxis=dict(
        title='Doses'
    ),
    yaxis2=dict(
        title='Rate %',
         anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis3=dict(
        title='Deaths',
         anchor="free",
        overlaying="y",
        side="right",
        position=1
    )
)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout(
    title_text="Vaccinations, Positivity Rate % & Deaths"
)

fig.show()

### Vaccinations, Positivity Rate & Hospitalizations

In [67]:
fig = go.Figure()

fig.add_traces([
    go.Bar(name='Received One Dose', x=data_df['Date'], y=data_df['New Received one dose']),
    go.Bar(name='Primary Vaccination', x=data_df['Date'], y=data_df['New Primary Vaccination']),
    go.Bar(name='Booster Doses', x=data_df['Date'], y=data_df['New Booster doses']),
    go.Bar(name='2nd Booster Doses', x=data_df['Date'], y=data_df['New 2nd Booster doses']),
    go.Bar(name='Omicron Booster Doses', x=data_df['Date'], y=data_df['New Omicron booster doses'])
])
fig.update_layout(barmode='stack')

fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['Positivity Rate'],
        mode='lines+markers',
        name='Positivity Rate',
        marker_color='black',
        yaxis='y2'
    )
)
fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['New Hospitalized'],
        mode='lines+markers',
        name='Hospitalizations',
        yaxis='y3'
    )
)
fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['New ICU'],
        mode='lines+markers',
        name='ICU',
        yaxis='y3'
    )
)

fig.update_layout(
    xaxis=dict(
        domain=[0, 0.95]
    ),
    yaxis=dict(
        title='Doses'
    ),
    yaxis2=dict(
        title='Rate %',
         anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis3=dict(
        title='Hospitalizations',
         anchor="free",
        overlaying="y",
        side="right",
        position=1
    )
)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout(
    title_text="Vaccinations, Positivity Rate % & Hospitalizations"
)

fig.show()

### Vaccinations, Recovered & Active Cases

In [68]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_traces([
    go.Bar(name='Received One Dose', x=data_df['Date'], y=data_df['New Received one dose']),
    go.Bar(name='Primary Vaccination', x=data_df['Date'], y=data_df['New Primary Vaccination']),
    go.Bar(name='Booster Doses', x=data_df['Date'], y=data_df['New Booster doses']),
    go.Bar(name='2nd Booster Doses', x=data_df['Date'], y=data_df['New 2nd Booster doses']),
    go.Bar(name='Omicron Booster Doses', x=data_df['Date'], y=data_df['New Omicron booster doses'])
])
fig.update_layout(barmode='stack')

fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['New Recovered'],
        mode='lines+markers',
        name='Recovered',
        marker_color='black'
    ),
    secondary_y=True
)
fig.add_trace(
    go.Scatter(
        x=data_df['Date'],
        y=data_df['Active Cases'],
        mode='lines+markers',
        name='Active Cases',
        marker_color='blue'
    ),
    secondary_y=True
)

fig.update_yaxes(title_text="Doses", secondary_y=False)
fig.update_yaxes(title_text="Cases", secondary_y=True)

fig.update_layout(
    title_text="Vaccinations, Recovered & Active Cases"
)

fig.show()